In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import flwr
import torch
import dotenv
import wandb
import os

from torch.optim.lr_scheduler import CosineAnnealingLR

from fl_g13.fl_pytorch import build_fl_dependencies

from fl_g13.fl_pytorch.editing import load_or_create_centralized_mask
from fl_g13.editing.masking import mask_dict_to_list

from fl_g13.fl_pytorch.datasets import reset_partition
from fl_g13.fl_pytorch.client_app import get_client_app
from fl_g13.fl_pytorch.server_app import get_server_app
from flwr.simulation import run_simulation

from fl_g13.architectures import BaseDino
from fl_g13.editing import SparseSGDM

c:\Users\ciovi\Desktop\coding\mldl\fl-g13\.venv\lib\site-packages\dockerpycreds\utils.py:1: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.spawn
2025-06-21 13:12:56.982 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ciovi\Desktop\coding\mldl\fl-g13


In [3]:
dotenv.load_dotenv()

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")

build_fl_dependencies()

# login by key in .env file
WANDB_API_KEY = dotenv.dotenv_values()["WANDB_API_KEY"]
wandb.login(key=WANDB_API_KEY)

Training on cuda
Flower 1.17.0 / PyTorch 2.7.1+cu128
'vision_transformer.py' already exists.
'utils.py' already exists.


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\ciovi\_netrc
wandb: Currently logged in as: tarantino-giovanbattista01 (stefano-gamba-social-politecnico-di-torino) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# FL CONFIG

In [4]:
CHECKPOINT_DIR = dotenv.dotenv_values()["CHECKPOINT_DIR"]

# Model config
## Model Hyper-parameters
head_layers = 3
head_hidden_size = 512
dropout_rate = 0.0
unfreeze_blocks = 1

## Training Hyper-parameters
batch_size = 128
lr = 1e-3
momentum = 0.9
weight_decay = 1e-5
T_max = 8
eta_min = 1e-5

# FL config
K = 100
C = 0.1
J = 8
num_rounds = 30
partition_type = 'shard'

## only for partition_type = 'shard'
num_shards_per_partition = 50

## Server App config
save_every = 5
fraction_fit = C  # Sample of available clients for training
fraction_evaluate = 0.1  # Sample 50% of available clients for evaluation
min_fit_clients = 10  # Never sample less than 10 clients for training
min_evaluate_clients = 5  # Never sample less than 5 clients for evaluation
min_available_clients = 10  # Wait until all 10 clients are available
device = DEVICE
client_batch_size = 16

# Adaptive Quorum strategy
strategy = 'quorum'
mask_strategy = 'sum'
quorum_increment = 6
quorum_update_frequency = 5
initial_quorum = 1
adaptive_quorum = True
initial_target_sparsity = 0.60
drift_threshold = 0.00041
quorum_patience = 4
force_quorum_update = 17

# model editing config
model_editing = True
mask_type = 'local'
sparsity = 0.7
num_calibration_rounds = 3
mask = None

## simulation run config
NUM_CLIENTS = 100
MAX_PARALLEL_CLIENTS = 10

total_rounds = 300
evaluate_each = 5

# Device settings

# When running on GPU, assign an entire GPU for each client
# Refer to Flower framework documentation for more details about Flower simulations
# and how to set up the `backend_config`
if device == "cuda":
    backend_config = {
        "client_resources": {
            "num_cpus": 1, 
            "num_gpus": 1
        }
    }
else:
    backend_config = {
        "client_resources": {
            "num_cpus": 1, 
            "num_gpus": 0
        }
    }

print(f"Training on {device}")

partition_name = 'iid' if partition_type == 'iid' else 'non_iid'
model_save_path = CHECKPOINT_DIR + f"/fl_dino_v4/{partition_name}/{num_shards_per_partition}_{J}"
model_config={
    "head_layers": 3,
    "head_hidden_size": 512,
    "dropout_rate": 0.0,
    "unfreeze_blocks": 0,
}

Training on cuda


In [5]:
from fl_g13.modeling import load_or_create

# Model
model, start_epoch = load_or_create(
    path=model_save_path,
    model_class=BaseDino,
    model_config=None,
    optimizer=None,
    scheduler=None,
    device=device,
    verbose=True,
)
model.to(DEVICE)

unfreeze_blocks = 12
model.unfreeze_blocks(unfreeze_blocks)
# optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)

# Create a dummy mask for SparseSGDM
# Must be done AFTER the model is moved to the device
init_mask = [torch.ones_like(p, device=p.device) for p in model.parameters()]

# Optimizer, scheduler, and loss function
optimizer = SparseSGDM(
    model.parameters(),
    mask=init_mask,
    lr=lr,
    momentum=0.9,
    weight_decay=1e-5
)
criterion = torch.nn.CrossEntropyLoss()
scheduler = CosineAnnealingLR(
    optimizer=optimizer,
    T_max=T_max,
    eta_min=eta_min
)

🔍 Loading checkpoint from /Users/ciovi/Desktop/coding/mldl/fl-g13/checkpoints/fl_dino_v4/non_iid/50_8\fl_fl_baseline_BaseDino_epoch_200_noniid_50_8.pth
📦 Model class in checkpoint: BaseDino
🔧 Model configuration: {'variant': 'dino_vits16', 'dropout_rate': 0.0, 'head_hidden_size': 512, 'head_layers': 3, 'num_classes': 100, 'unfreeze_blocks': 0, 'activation_fn': 'GELU', 'pretrained': True}


Using cache found in C:\Users\ciovi/.cache\torch\hub\facebookresearch_dino_main
Using cache found in C:\Users\ciovi/.cache\torch\hub\facebookresearch_dino_main


➡️ Moved model to device: cuda
✅ Loaded checkpoint from /Users/ciovi/Desktop/coding/mldl/fl-g13/checkpoints/fl_dino_v4/non_iid/50_8\fl_fl_baseline_BaseDino_epoch_200_noniid_50_8.pth, resuming at epoch 201


In [6]:
file_name = CHECKPOINT_DIR + '/masks/' + f'{num_shards_per_partition}_{J}_{mask_strategy}_mask_{mask_type}_{sparsity}_{num_calibration_rounds}.pth'

sum_mask, _ = load_or_create_centralized_mask(
    model = model,
    strategy = mask_strategy,
    aggregation_fn = None,
    client_partition_type = partition_type,
    client_num_shards_per_partition = num_shards_per_partition,
    client_local_steps = J,
    
    sparsity = sparsity,
    mask_type = mask_type,
    mask_rounds = num_calibration_rounds,
    
    file_name = file_name,
    verbose = True
)
sum_mask = mask_dict_to_list(model, sum_mask)

[CENTR_MASK] Found /Users/ciovi/Desktop/coding/mldl/fl-g13/checkpoints/masks/50_8_sum_mask_local_0.7_3.pth. Loading mask from memory


# Simulation

In [ ]:
reset_partition()

checkpoint_dir = CHECKPOINT_DIR + f"/fl_dino_v4/{partition_name}/dq{'_A' if adaptive_quorum else '_L'}_{num_shards_per_partition}_{J}_{strategy}_{mask_type}_{sparsity}_{num_calibration_rounds}"
os.makedirs(checkpoint_dir, exist_ok=True)

## Wandb config
use_wandb = True
name = f"DQ_{'A' if adaptive_quorum else 'L'}_{num_shards_per_partition}_{J}_{mask_type}_{sparsity}_{num_calibration_rounds}"
print(name)

wandb_config = {
    # wandb param
    'name': name,
    'project_name': "Dynamic-Quorum",
    
    # model config
    "lr": lr,
    "momentum": momentum,
    
    # FL config
    'K': K,
    'C': C,
    'J': J,
    'partition_type': partition_type,
    "fraction_fit": fraction_fit,
    
    # model editing config
    'mask_type': mask_type,
    'sparsity': sparsity,
    'num_calibration_rounds': num_calibration_rounds,
    
    # strategy config
    'adaptive_quorum': adaptive_quorum,
    'initial_quorum': initial_quorum,
    'initial_target_sparsity': initial_target_sparsity,
    'quorum_update_frequency': quorum_update_frequency,
    'quorum_increment': quorum_increment,
    'drift_threshold': drift_threshold,
    'quorum_patience': quorum_patience,
    'force_quorum_update': force_quorum_update
}

client = get_client_app(
        model=model,
        optimizer=optimizer,
        criterion=criterion,
        device=device,
        partition_type=partition_type,
        local_epochs=1,
        local_steps=J,
        batch_size=batch_size,
        num_shards_per_partition=num_shards_per_partition,
        scheduler=scheduler,
        verbose=0,
        mask=mask,
        model_editing_batch_size=client_batch_size,
        mask_func=None,
        strategy=strategy,
    )

compute_round = total_rounds + 1 - start_epoch

server = get_server_app(
    global_mask = sum_mask,
    num_total_clients = NUM_CLIENTS,
    adaptive_quorum = adaptive_quorum,
    initial_target_sparsity = initial_target_sparsity,
    quorum_increment = quorum_increment,
    quorum_update_frequency = quorum_update_frequency,
    initial_quorum = initial_quorum,
    drift_threshold = drift_threshold,
    quorum_patience = quorum_patience,
    force_quorum_update = force_quorum_update,
    
    checkpoint_dir=checkpoint_dir,
    model_class=model,
    optimizer=optimizer,
    criterion=criterion,
    scheduler=scheduler,
    num_rounds=compute_round,
    fraction_fit=fraction_fit,
    fraction_evaluate=fraction_evaluate,
    min_fit_clients=min_fit_clients,
    min_evaluate_clients=min_evaluate_clients,
    min_available_clients=min_available_clients,
    device = device,
    use_wandb=use_wandb,
    wandb_config = wandb_config,
    save_every=save_every,
    prefix = name,
    evaluate_each=evaluate_each,
    model = model,
    start_epoch=start_epoch,
    strategy=strategy
)

DQ_A_50_8_local_0.7_3


In [8]:
try:
    run_simulation(
        server_app=server,
        client_app=client,
        num_supernodes=NUM_CLIENTS,
        backend_config=backend_config
    )
except:
    pass

wandb.finish()

Using strategy 'DynmicQuorum'


c:\Users\ciovi\Desktop\coding\mldl\fl-g13\.venv\lib\site-packages\wandb\analytics\sentry.py:259: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}  # noqa


INFO :      [DQ] ADAPTIVE mode enabled. Quorum: 9; Drift threshold: 0.00041
INFO :      Starting Flower ServerApp, config: num_rounds=100, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
Eval progress: 100%|██████████| 313/313 [00:23<00:00, 13.45batch/s]
INFO :      [Round 0] Centralized Evaluation - Loss: 1.2866, Metrics: {'centralized_accuracy': 0.632}
INFO :      initial parameters (loss, other metrics): 1.2865915495557145, {'centralized_accuracy': 0.632}
INFO :      
INFO :      [ROUND 1]
INFO :      [Round 1 DQ] Generated global mask with sparsity: 0.6027
INFO :      configure_fit: strategy sampled 10 clients (out of 100)
(ClientAppActor pid=20348) 2025-06-21 13:13:36.317 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ciovi\Desktop\coding\mldl\fl-g13
(ClientAppActor pid=20348) c:\Users\ciovi\Desktop\coding\mldl\fl-g13\.venv\lib\site-p

💾 Saved checkpoint at: /Users/ciovi/Desktop/coding/mldl/fl-g13/checkpoints/fl_dino_v4/non_iid/dq_A_50_8_quorum_local_0.7_3\fl_DQ_A_50_8_local_0.7_3_BaseDino_epoch_205.pth


Eval progress: 100%|██████████| 313/313 [00:22<00:00, 13.98batch/s]
INFO :      [Round 5] Centralized Evaluation - Loss: 0.9588, Metrics: {'centralized_accuracy': 0.7107}
INFO :      fit progress: (5, 0.9588400863420469, {'centralized_accuracy': 0.7107}, 343.1027322000009)
INFO :      configure_evaluate: strategy sampled 10 clients (out of 100)
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [ROUND 6]
INFO :      configure_fit: strategy sampled 10 clients (out of 100)
INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      [Round 6] Avg Drift: 0.0005 | Relative Drift: 0.0000
INFO :      fit progress: (6, {}, {}, 405.4978303000007)
INFO :      configure_evaluate: strategy sampled 10 clients (out of 100)
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [ROUND 7]
INFO :      configure_fit: strategy sampled 10 clients (out of 100)
INFO :      aggregate_fit: received 10 results and 0 fai

...

💾 Saved checkpoint at: /Users/ciovi/Desktop/coding/mldl/fl-g13/checkpoints/fl_dino_v4/non_iid/dq_A_50_8_quorum_local_0.7_3\fl_DQ_A_50_8_local_0.7_3_BaseDino_epoch_295.pth


Eval progress: 100%|██████████| 313/313 [00:22<00:00, 13.71batch/s]
INFO :      [Round 95] Centralized Evaluation - Loss: 0.5876, Metrics: {'centralized_accuracy': 0.8172}
INFO :      fit progress: (95, 0.587551694018201, {'centralized_accuracy': 0.8172}, 6259.107483)
INFO :      configure_evaluate: strategy sampled 10 clients (out of 100)
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [ROUND 96]
INFO :      configure_fit: strategy sampled 10 clients (out of 100)
INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      [Round 96] Avg Drift: 0.0003 | Relative Drift: 0.0000
INFO :      fit progress: (96, {}, {}, 6317.5718896)
INFO :      configure_evaluate: strategy sampled 10 clients (out of 100)
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [ROUND 97]
INFO :      configure_fit: strategy sampled 10 clients (out of 100)
INFO :      aggregate_fit: received 10 results and 0 failures


💾 Saved checkpoint at: /Users/ciovi/Desktop/coding/mldl/fl-g13/checkpoints/fl_dino_v4/non_iid/dq_A_50_8_quorum_local_0.7_3\fl_DQ_A_50_8_local_0.7_3_BaseDino_epoch_300.pth


Eval progress: 100%|██████████| 313/313 [00:22<00:00, 13.73batch/s]
INFO :      [Round 100] Centralized Evaluation - Loss: 0.5786, Metrics: {'centralized_accuracy': 0.8188}
INFO :      fit progress: (100, 0.5785657061983983, {'centralized_accuracy': 0.8188}, 6572.3364781)
INFO :      configure_evaluate: strategy sampled 10 clients (out of 100)
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 100 round(s) in 6580.56s
INFO :      	History (loss, distributed):
INFO :      		round 1: 1.1573746860027314
INFO :      		round 2: 1.0152924239635468
INFO :      		round 3: 0.9928029119968415
INFO :      		round 4: 0.9133639335632324
INFO :      		round 5: 0.9029825508594513
INFO :      		round 6: 0.8810269296169281
INFO :      		round 7: 0.965679121017456
INFO :      		round 8: 0.8628397941589355
INFO :      		round 9: 0.883795416355133
INFO :      		round 10: 0.9151702225208282
INFO :      		round 11: 0.889605927467346

centralized_accuracy,▁▄▅▅▆▆▆▇▇▇▇▇▇████████
centralized_eval_loss,█▅▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
decentralized_avg_eval_accuracy,▁▃▄▄▄▃▄▅▅▆▆▆▆▆▇▇▅▆▇▆▆▇▇▆▇█▇▆▇▇▇▇▇▇▇█▇█▇▇
decentralized_avg_eval_loss,█▇▇█▇▆▆▆▆▅▄▄▄▄▄▅▄▅▄▃▄▄▃▃▃▂▃▂▃▂▂▂▃▂▃▁▂▁▃▂
decentralized_avg_train_accuracy,▁▁▃▃▄▄▄▅▅▆▅▅▆▆▆▆▆▆▇▇▇▇█▇▇▇▇█▇███████████
decentralized_avg_train_loss,██▅▆▅▅▄▅▄▄▃▃▃▃▃▃▃▃▂▂▂▃▂▂▂▁▂▂▂▂▂▁▂▁▁▂▁▁▁▂
mask_sparsity,▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▇█
quorum,▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▅▅▅▆▆▆▇▇▇▇█
centralized_accuracy,0.8188
centralized_eval_loss,0.57857
decentralized_avg_eval_accuracy,0.816
